I used pretrained resnet-50 and inception-v3 from https://github.com/Lasagne/Recipes. I obtained vectors from layer before softmax. These vectors was concatenated (4096 dimension vector as a result) and used as input for a simple two layer neural network. Parametres for this network was choosen by testing on validation dataset.

In [1]:
import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu7')

Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GTX TITAN X (0000:04:00.0)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 7: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [2]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import time

### Load data

In [3]:
resnet_data = np.load('data_resnet.npz')
inception_data = np.load('data_inception.npz')

In [4]:
y_train = np.zeros(25000, dtype=np.int64)
y_train[:12500] = 1

In [5]:
X_train = np.hstack([resnet_data['X_train'], inception_data['X_train']])
X_test = np.hstack([resnet_data['X_test'], inception_data['X_test']])

In [6]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

### Model

In [8]:
net = {}
net['input'] = lasagne.layers.InputLayer((None, X_train.shape[1]))
net['dropout'] = lasagne.layers.DropoutLayer(net['input'], p=0.4)
net['dense_1'] = lasagne.layers.DenseLayer(
    net['dropout'], 
    512, 
    nonlinearity=lasagne.nonlinearities.LeakyRectify())
net['dense_2'] = lasagne.layers.DenseLayer(net['dense_1'], 2, nonlinearity=None)
net['prob'] = lasagne.layers.NonlinearityLayer(net['dense_2'], lasagne.nonlinearities.softmax)

In [9]:
input_X = T.fmatrix('X')
target_y = T.vector('y', dtype='int64') 

In [10]:
predicted_y = lasagne.layers.get_output(net['prob'], input_X)
evaluated_y = lasagne.layers.get_output(net['prob'], input_X, deterministic=True)

loss = lasagne.objectives.categorical_crossentropy(predicted_y, target_y).mean()
train_accuracy = lasagne.objectives.categorical_accuracy(predicted_y, target_y).mean()

accuracy = lasagne.objectives.categorical_accuracy(evaluated_y, target_y).mean()

In [11]:
weights = lasagne.layers.get_all_params(net['prob'], trainable=True)
lr = theano.shared(np.array(4 * 1e-4, dtype=np.float32))
updates = lasagne.updates.adam(loss, weights, learning_rate=lr)

In [12]:
train_fun = theano.function([input_X, target_y], [loss, train_accuracy], updates=updates)
predict_fun = theano.function([input_X], evaluated_y)

### Train

In [13]:
import tqdm

In [14]:
def iterate_minibatches(X, y, batchsize):
    n_samples = X.shape[0]
    perm = np.arange(n_samples)
    np.random.shuffle(perm)
    for start in tqdm.tqdm(xrange(0, n_samples, batchsize)):
        end = min(start+batchsize, n_samples)
        batch_idx = perm[start:end]
        yield X[batch_idx], y[batch_idx]

In [15]:
import time

num_epochs = 25

batch_size = 128

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1
        
    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        predicted_y = predict_fun(inputs)
        val_acc += (predicted_y.argmax(axis=1) == targets).mean()
        val_batches += 1
    if (epoch + 1) % 5 == 0:
        lr.set_value(lr.get_value() / 4)
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

  5%|▌         | 9/176 [00:00<00:01, 84.74it/s]

Epoch 1 of 25 took 2.017s
  training loss (in-iteration):		5.469642
  train accuracy:		97.50 %
  validation accuracy:		99.22 %


  6%|▌         | 10/176 [00:00<00:01, 99.66it/s]

Epoch 2 of 25 took 2.048s
  training loss (in-iteration):		0.724843
  train accuracy:		98.89 %
  validation accuracy:		99.38 %


  6%|▌         | 10/176 [00:00<00:01, 94.70it/s]

...

Epoch 23 of 25 took 1.945s
  training loss (in-iteration):		0.041061
  train accuracy:		99.48 %
  validation accuracy:		99.50 %


  5%|▌         | 9/176 [00:00<00:02, 83.13it/s]

Epoch 24 of 25 took 1.940s
  training loss (in-iteration):		0.039904
  train accuracy:		99.53 %
  validation accuracy:		99.50 %


100%|██████████| 20/20 [00:00<00:00, 592.84it/s]

Epoch 25 of 25 took 1.925s
  training loss (in-iteration):		0.040023
  train accuracy:		99.53 %
  validation accuracy:		99.53 %


### Create submission

In [16]:
y_test = predict_fun(X_test).argmax(axis=1)

In [17]:
import pandas as pd

In [18]:
test_files = [('%d.jpg' % (i)) for i in range(1000)]
labels = ['dog' if y else 'cat' for y in y_test]
df = pd.DataFrame({'file' : test_files, 'label' : labels})
df.to_csv('submission.csv', index=False)